In [ ]:
# Use a small dataset (e.g., CIFAR-10) and preprocess it for transfer learning.

from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
import tensorflow as tf

# Load the CIFAR-10 dataset
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

# Normalize pixel values to range [0, 1]
X_train, X_test = X_train / 255.0, X_test / 255.0

# Convert labels to one-hot encoding
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

# Resize images to fit the pre-trained model's input size
X_train = tf.image.resize(X_train, (224, 224))
X_test = tf.image.resize(X_test, (224, 224))

print("Training data shape:", X_train.shape)
print("Test data shape:", X_test.shape)

In [ ]:
# Load a pre-trained model like VGG16 or MobileNet without the top layers to adapt it for a new task.

from tensorflow.keras.applications import VGG16

# Load the pre-trained VGG16 model without the top layers
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the base model's layers
base_model.trainable = False

print("Base model layers frozen:", not base_model.trainable)

In [ ]:
# Add custom layers to the pre-trained base model for classifying CIFAR-10 images.

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout

# Build the transfer learning model
model = Sequential([
    base_model,
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(10, activation='softmax')  # Output layer for 10 classes
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

In [ ]:
# Train the model on the new dataset while keeping the pre-trained layers frozen.

history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test), verbose=2)

In [ ]:
# Unfreeze some layers of the base model to fine-tune it for the specific dataset.

# Unfreeze the last few layers of the base model
base_model.trainable = True
for layer in base_model.layers[:-4]:  # Keep the first few layers frozen
    layer.trainable = False

# Recompile the model with a lower learning rate for fine-tuning
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5), loss='categorical_crossentropy', metrics=['accuracy'])

# Fine-tune the model
history_fine = model.fit(X_train, y_train, epochs=5, batch_size=32, validation_data=(X_test, y_test), verbose=2)

In [ ]:
# Evaluate the fine-tuned model on the test set and predict new samples.

# Evaluate test accuracy
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
print("Test Accuracy after Fine-Tuning:", test_accuracy)

# Predict a single image
import numpy as np
import matplotlib.pyplot as plt

sample_image = np.expand_dims(X_test[0], axis=0)  # Add batch dimension
prediction = model.predict(sample_image)
predicted_class = np.argmax(prediction)

# Plot the sample image with predicted class
plt.imshow(X_test[0])
plt.title(f"Predicted Class: {predicted_class}")
plt.axis('off')
plt.show()